# Generador de archivos para experimentar

# Pasaje de parametros (luego, correr ./run_all.sh)

In [1]:
# Seteo nombre del directorio en el cual voy a guardar todos los experimentos
experiment_name = 'exp_pedro_1'

In [2]:
%%sh -s "$experiment_name"
rm -r "$1"
mkdir "$1"
#mkdir "$1"/src
#cp -r ../src_base/* "$1"/src

rm: cannot remove 'exp_pedro_1': No such file or directory


In [3]:
# Parametros

# TODO : permitir modificar el 'delta' del modelo de opinion (Cell-Devs)
# TODO : permitir setear el P0 (porcentaje de indefinidos) en el estado inicial de las celdas

N = 10
M = 10
numGroups = 5
percentageOfShockedCellsPerShocker = 0.4

modelName  = 'opinion'
atomicName = 'shocker'
atomicClass= 'Shocker'

# Seteo la cantidad de condiciones iniciales que quiero testear
nInitialConditions = 1

# Seteo el tiempo de simulacion
stop_time = "00:00:55:00:000"

# Seteo delay del cell-devs para que guarde una relacion que se condiga con la evolucion del modelo
# de arriba
delayValues = [83]

# Input parameters
input_parameters = {
    'Betaa' : 0.015, 
    'rateInterestOnLoans' : 0.04, 
    'Alphaa' : 0.025, 
    'employmentRateStable' : 10,
    'deltaKReal' : 0.05, 
    'velocityOfMoney' : 3,
    'piZ' : 0.04,
    'employmentRateZero' : 0.6,
    'piS' : 10
}

# 0 => los shocks vienen dados de acuerdo a thresholds que cruzan valores obtenidos a partir de la relacion
#      de las variables durante la evolucion del sistema
# 1 => los shocks vienen dados en ciertos instantes de tiempo, definidos en el events file
shockerTypeValues = [0,1]

# Cuando shockerTypeValue == 1, en el events file mando los tiempos en los que hay que shockear
shocksActivationTime = ['00:00:10:00', '00:00:20:00', '00:00:30:00', '00:00:45:00']

# 1 = envio shocks negativo / negativo / negativo / negativo, segun  limites 80 / 60 / 40 / 20 para Neutralists
# => envia 4 shocks en total
# 2 = envio shocks positivo / positivo / positivo / positivo, segun limites 80 / 60 / 40 / 20 para Neutralists
# => envia 4 shocks en total
shockCriteriaValues = [1]

# InitialConditions: [%negative, %neutral, %positive]
initialConditionsValues = [
    [0.2, 0.6, 0.2],
    [0.1, 0.8, 0.1],
    #[0.2, 0.4, 0.4],
    #[0.4, 0.4, 0.2],
    #[0.4, 0.2, 0.4],
    [0.8, 0.1, 0.1]
]

# Harcodeo reglas extra (reaccion ante 'SHOCKS')
rules = [
    '{ uniform(-3, -2) } 0 { portValue(thisPort) = 5 }',
    '{ uniform(-1, 1) } 0 { portValue(thisPort) = 6 }',
    '{ uniform(2, 3) } 0 { portValue(thisPort) = 7 }',
    '{ (0,0,0) } 0 { portValue(thisPort) = 8 }',
    '{ (0,0,0) } 0 { t }'
]

# Generacion de experimentos

In [4]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import numpy as np
import random as rd
import numpy as np
from random import shuffle

PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates-experimentos')),
    trim_blocks=False)

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

def generateRegFile(template, atomicClass):
    templateReg = template
    context = {
        'atomicClass' : atomicClass,
        'atomicClassConstant' : atomicClass.upper()
    }
    regFile = render_template(templateReg, context)
    return regFile

def generateMAFile(template, valFile, N, M, atomicos, inputs, links_internal, links_external, 
                   inputs_external, links_external_atomic, links_external_transition, rules):
    templateMA = template
    context = { 
        'N' : N,
        'M' : M,
        'valFile'  : valFile,
        'atomicos' : atomicos,
        'inputs_internal'   : inputs, 
        'inputs_external'   : inputs_external,
        'links_external_atomic' : links_external_atomic,
        'links_internal'    : links_internal, 
        'links_external'    : links_external,
        'links_external_transition' : links_external_transition,
        'rules' : rules,
        'delayValue' : "{{delayValue}}"
    }
    maFile = render_template(templateMA, context)
    return maFile

def generateAtomicFile(templateH, templateCPP, atomicClass, outPorts, numberOfChosenOutputPorts):
    context = {
        'atomicClass': atomicClass,
        'atomicClassConstant' : atomicClass.upper(),
        'outPorts'   : outPorts,
        'numberOfOutputPorts' : len(outPorts),
        'numberOfChosenOutputPorts' : numberOfChosenOutputPorts
    }
    hFile   = render_template(templateH, context)
    cppFile = render_template(templateCPP, context)
    return {
        'h'   : hFile,
        'cpp' : cppFile
    }
    return 0

def generateInputAndLinks(N,M,numGroups,modelName,atomicName,atomicClass):
    # Celdas
    cells  = [(n,m) for n in range(0,N) for m in range(0,M)]
    assign = []
    val = 0
    while len(cells) > 0:
        i = rd.randint(0,len(cells)-1)
        assign.append((cells[i][0], cells[i][1], val))
        cells.pop(i)
        val = (val + 1) % numGroups
    
    # Inputs + Links external
    inputs_external = ['in' + str(i) for i in range(0,numGroups)]
    links_external_atomic = ['link : in' + str(i) + ' ' + 'in@' + atomicName + str(i) 
                             for i in range(0,numGroups)]

    # Mapeo conexiones de atomicos 0..numGroups a las celdas correspondientes
    outputsPerAtomic = int(N * M / numGroups)
    assert(outputsPerAtomic * numGroups == N * M)
    
    outputsAtomicos = [[] for i in range(0,numGroups)]
    numOutAssigned  = [0] * numGroups 
    
    inputsModelo  = []
    linksInternos = []
    linksExternos = []
    links_external_transition = []
    for assignation in assign:
        inputCellNumber = str(assignation[0]) + str(assignation[1])
        
        # Inputs modelo
        inputsModelo.append('in' + inputCellNumber)

        # Links internos
        linksInternos.append('link : in' + inputCellNumber + ' in' +
                             str(assignation[0]) + str(assignation[1]) +
                             '@' + modelName + 
                             '(' + str(assignation[0]) + ',' + str(assignation[1]) + ',0)')
        
        # Links external transition
        links_external_transition.append('PortInTransition : ' + 'in' + 
                             str(assignation[0]) + str(assignation[1]) +            
                             '@' + modelName + 
                             '(' + str(assignation[0]) + ',' + str(assignation[1]) + ',0)' +
                             ' ' + 'shockExterno')
        
        # Links externos
        outputAtomicNumber = str(assignation[2])
        linksExternos.append('link : out' + str(numOutAssigned[assignation[2]]) + 
                             '@' + atomicName + str(assignation[2]) + ' ' + 'in' + inputCellNumber + 
                             '@' + modelName)

        # Mapa Outputs <==> Atomicos
        outputsAtomicos[assignation[2]].append('out' + str(numOutAssigned[assignation[2]]))
    
        # Instancias de atomicos
        atomicos = [atomicName + str(i) + '@' + atomicClass for i in range(0,numGroups)]
        
        # actualizo valores
        numOutAssigned[assignation[2]] += 1
        
    return {'inputsModelo'  : sorted(inputsModelo), 
            'linksInternos' : sorted(linksInternos),
            'linksExternos' : sorted(linksExternos),
            'atomicos'      : atomicos,
            'outputsAtomicos' : outputsAtomicos[0],
            'inputs_external' : inputs_external,
            'links_external_atomic' : links_external_atomic,
            'links_external_transition' : links_external_transition}

# Genero archivos para correr tests

In [5]:
# Genero archivo .ma
inputsLinks = generateInputAndLinks(N,M,numGroups,modelName,atomicName,atomicClass)
inputsModelo    = inputsLinks['inputsModelo']
links_externos  = inputsLinks['linksExternos']
links_internos  = inputsLinks['linksInternos']
atomicos        = inputsLinks['atomicos']
outputsAtomicos = inputsLinks['outputsAtomicos']
inputs_external = inputsLinks['inputs_external']
links_external_atomic = inputsLinks['links_external_atomic']
links_external_transition = inputsLinks['links_external_transition']

# PARAM : a que porcentaje de las celdas impacta cada shock?
outPorts = outputsAtomicos
nChosenOutports = int(len(outPorts) * percentageOfShockedCellsPerShocker)


# Guardo archivo .ma de la conexion
maFilename = 'templates-experimentos/template-opinion-goodwin-minsky-conexion-for-experiments.ma'
mafile = generateMAFile('template-opinion-goodwin-minsky-conexion.ma', 'valfile.val', N, M,
                        atomicos, inputsModelo, 
                        links_internos, links_externos, inputs_external, links_external_atomic,
                        links_external_transition, rules)
folder = './'
with open(folder + maFilename, 'w') as f:
        f.write(mafile)

# Guardo archivo .ma del modelo de opinion solamente
maFilename2 = experiment_name + '/opinion.ma'
mafile2 = generateMAFile('template-opinion.ma', 'valfile.val', N, M,
                        atomicos, inputsModelo, 
                        links_internos, links_externos, inputs_external, links_external_atomic,
                        links_external_transition, rules)

with open(folder + maFilename2, 'w+') as f:
        f.write(mafile2)

# Genero archivos .h y .cpp 
#folder = experiment_name + '/src/'
#atomic = generateAtomicFile('template-shocker.h', 'template-shocker.cpp', 
#                                atomicClass, outPorts, nChosenOutports)

#with open(folder + atomicClass + '.h', 'w') as f:
#    hFile   = atomic['h']
#    f.write(hFile)

#with open(folder + atomicClass + '.cpp', 'w') as f:
#    cppFile = atomic['cpp']
#    f.write(cppFile)

In [6]:
def generateInitialValues(initialConditionValues, N, M):
    pnegative, pneutral, ppositive = initialConditionValues[0], initialConditionValues[1], initialConditionValues[2]
    negative, neutral = int(N*M*pnegative), int(N*M*pneutral)
    positive = N*M - neutral - negative
    vpositive = [np.random.uniform(1,3) for i in range(positive)]
    vneutral  = [np.random.uniform(-1,1) for i in range(neutral)]
    vnegative = [np.random.uniform(-3,-1) for i in range(negative)]
    v = vpositive + vneutral + vnegative
    shuffle(v)
    res = []
    for i in range(N):
        for j in range(M):
            res.append('(' + str(i) + ',' + str(j) + ',0)=' + str(v[M*i+j]))
    return res

templateMA     = 'template-opinion-goodwin-minsky-conexion-for-experiments.ma'
templateEV     = 'template-events.ev'
templateVAL    = 'template-valfile.val'
templateMACROS = 'template-macros.inc'
templateBASH   = 'template-run-bash.sh'
templateRUNALL = 'template-run-all.sh'

nExperiment = 1
for delayValue in delayValues:
    for shockCriteriaValue in shockCriteriaValues:
        for shockerTypeValue in shockerTypeValues:
            for initialConditionValues in initialConditionsValues:
                dir_name = experiment_name + '/' + str(delayValue) + \
                    '_' + str(shockerTypeValue) + \
                    '_' + str(shockCriteriaValue) + \
                    '_' + str(initialConditionValues[0]) + '_' + str(initialConditionValues[1]) + \
                    '_' + str(initialConditionValues[2])
                
                # Genero archivos .h y .cpp 
                folder = dir_name + '/src/'
                os.makedirs(os.path.dirname(folder))
                from distutils.dir_util import copy_tree
                copy_tree("../src_base", folder)
                
                atomic = generateAtomicFile('template-shocker.h', 'template-shocker.cpp', 
                                                atomicClass, outPorts, nChosenOutports)

                with open(folder + atomicClass + '.h', 'w+') as f:
                    hFile   = atomic['h']
                    f.write(hFile)

                with open(folder + atomicClass + '.cpp', 'w+') as f:
                    cppFile = atomic['cpp']
                    f.write(cppFile)
                
                # Genero experimentos
                context = {
                    'delayValue' : delayValue,
                    'shockCriteriaValue' : shockCriteriaValue,
                    'shockerTypeValue' : shockerTypeValue,
                    'input_parameters' : input_parameters,
                    'shocksActivationTime' : shocksActivationTime,
                    'stop_time' : stop_time
                }

                # Generate directory if not exists
                if not os.path.exists(dir_name):
                    os.makedirs(dir_name)

                with open(dir_name + '/opinion-combined-model.ma', 'w') as f:
                    maFile = render_template(templateMA, context)
                    f.write(maFile)
                with open(dir_name + '/events.ev', 'w') as f:
                    evFile = render_template(templateEV, context)
                    f.write(evFile)
                with open(dir_name + '/macros.inc', 'w') as f:
                    macrosFile = render_template(templateMACROS, context)
                    f.write(macrosFile)

                # Generate bash for running experiments
                with open(dir_name + '/run.sh', 'w') as f:
                    bashFile = render_template(templateBASH, context)
                    f.write(bashFile)

                for experiment in range(0,nInitialConditions): 
                    # Generate context
                    context['values'] = generateInitialValues(initialConditionValues, N, M)
                    # Generate file
                    with open('{0}/valfile_exp_{1:02d}.val'.format(dir_name, experiment), 'w') as f:
                        valFile = render_template(templateVAL, context)
                        f.write(valFile)
                    nExperiment += 1
with open(experiment_name + '/run_all.sh', 'w') as f:
    bash_run_all = render_template(templateRUNALL, context)
    f.write(bash_run_all)